<a href="https://colab.research.google.com/github/chetan015/deep-learning/blob/master/Assignment-13/Assignment-13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ResNet18 Architecture implementation with Cyclic Learning Rate policy and training on CIFAR-10 dataset.
Reached 90.49% validation accuracy in 12 epochs, 810 seconds; 93% validation accuracy in 104 epochs, 7008 seconds.

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import numpy as np
import time, math
# Progress bar widget
from tqdm.notebook import tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# Enables tensorflow to execute and return results to python immediately, as opposed to adding to a graph to be executed later
tf.enable_eager_execution()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Define constants/parameters
BATCH_SIZE = 128 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.01 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 300 #@param {type:"integer"}


https://mc.ai/tutorial-1-cifar10-with-google-colabs-free-gpu%E2%80%8A-%E2%80%8A92-5/

In [0]:
# Used to initialize kernel weights
# The last dimension of shape defines the number of kernels. So the kernel
# Weights are initialized for all but last dimension
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  # Takes the product of all but last dimensions
  fan = np.prod(shape[:-1])
  # Bound is set to 1/sqrt of product
  # For a 5x5 kernel, prod is 25, bound is 1/5 = 0.2
  bound = 1 / math.sqrt(fan)
  # Outputs a random value from a uniform distribution in the range
  # [-lowerbound,upperbound) = [-0.2,0.2)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
# Define a class ConvBN, a subclass of keras.Model
# This basically creates the framework/blueprint of
# a convolution block with 3x3 convolution,
# followed by Dropout, Batch Normalization,  and finally ReLU activation
# OO approach to building models, facilitating reuse, compact and easy to 
# understand structure
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    # Initialize conv, bn, drop layers. Add as many filters as the number of channels required in output (c_out)
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.drop = tf.keras.layers.Dropout(0.05)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.drop(self.conv(inputs))))

In [0]:
# Define a class For ResNet Block
# Aids in creation of ResNet block, which involves a convolution block, 
# followed by maxpooling. If there is a skip connection
class ResBlk(tf.keras.Model):
  # Input has number of channels out, and pool 
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
# Finally Defining the entire Model
# 1. Convolution Block
# 2. ResNet Block
# 3. ResNet Block
# 4. ResNet Block
# 5. GAP
# 6. Dense Layer
# 7. Softmax

class Resnet18(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool,res = True)
    self.blk3 = ResBlk(c*4, pool,res = True)
    self.blk4 = ResBlk(c*8, pool, res = True)
    self.avg_pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.avg_pool(self.blk4(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x))))))
    h = self.linear(h) * self.weight
    # Computes sparse softmax cross entropy between logits and labels
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    # Sums across all dimensions to get the total loss
    loss = tf.reduce_sum(ce)
    # Get the predicted label as the logit with max value, compare for equality
    # Cast to float, sum to get total correct predictions
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [0]:
# Load the training dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)

# Y train and test are originally 2D arrays. There are as many rows as there are
# images, and each row has an an array with a single element - the label
# Reshape it into 1D array
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

# Setting mean and standard deviation of the dataset as provided in the assignment
train_mean =np.array([0.4914, 0.4822, 0.4465])
train_std = np.array([0.2023, 0.1994, 0.2010])
# Normalize and padding by 4 functions 
normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')
# Pad the images, extend axes 2 and 3 by four values on each side
# Basically pad each image with 4 pixels on each side
print(x_train.shape)
x_train = normalize(pad4(x_train))
x_test = normalize(x_test)
print(x_train.shape)

170500096/170498071 [==============================] - 13s 0us/step
(50000, 32, 32, 3)
(50000, 40, 40, 3)


In [0]:
# Cyclic Learning Rate Function
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops

def clr(global_step, step_size = 500., max_lr = 0.1, learning_rate=0.01):
    
    learning_rate = ops.convert_to_tensor(learning_rate, name="learning_rate")
    dtype = learning_rate.dtype
    global_step = math_ops.cast(global_step, dtype)
    step_size = math_ops.cast(step_size, dtype)

    double_step = math_ops.multiply(2., step_size)
    global_div_double_step = math_ops.divide(global_step, double_step)
    cycle = math_ops.floor(math_ops.add(1., global_div_double_step))
    #print (cycle)
    # computing: x = abs( global_step / step_size – 2 * cycle + 1 )
    double_cycle = math_ops.multiply(2., cycle)
    global_div_step = math_ops.divide(global_step, step_size)
    tmp = math_ops.subtract(global_div_step, double_cycle)  
    x = math_ops.abs(math_ops.add(1., tmp))

    # computing: clr = learning_rate + ( max_lr – learning_rate ) * max( 0, 1 - x )
    a1 = math_ops.maximum(0., math_ops.subtract(1., x))
    a2 = math_ops.subtract(max_lr, learning_rate)
    clr = math_ops.multiply(a1, a2)
    #print (math_ops.add(clr, learning_rate))
    return math_ops.add(clr, learning_rate)

In [0]:
# Create the model
model = Resnet18()
# Calculate number of batches per epoch
batches_per_epoch = len_train//BATCH_SIZE + 1
STEP_SIZE = 500 # 100 batchs = 1 step, 200 batches = 1 cycle
# Define Learning rate scheduler function
# Gives the value at t of the function interpolated between (0,0), 
# ((Epochs+1)//5,LR)) and (Epochs,0)  /\ The peak LR is at (Epochs+1//5) 
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
# Keep track of number of batches seen. Returns a tensor
global_step = tf.train.get_or_create_global_step()
# Computes the value t based on the current global step/batchesperepoch / Batch Size
# lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
lr_func = lambda: clr(global_step, STEP_SIZE, LEARNING_RATE, LEARNING_RATE/10)
# Define custom optimizer, which utilizes the learning rate calculation function
# and does sgd with nesterov momentum update
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
# Data aug first takes a random 32x32 crop of the 40x40 image, then performs random horizontal flip
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)


In [0]:
best_accuracy = 0.0
# Start training
t = time.time()
# Create the test dataset
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)
# Train the network for #EPOCHS
for epoch in range(EPOCHS):

  train_loss = test_loss = train_acc = test_acc = 0.0
  # Create train dataset, apply data augmentation and shuffle to the batch
  # Use prefetch to load the next batch while the model is training on current batch
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
  # Set learning phase: 0 for test, 1 for train
  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      # Forward Propagation
      loss, correct = model(x, y)
    # Backpropagation
    # Get trainable params, compute gradients
    # Apply gradients, carry out weight updates
    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()
  
  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
  val_accuracy = test_acc / len_test
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)
  if best_accuracy < val_accuracy:
    print("Best accuracy improved from ", best_accuracy, ' to: ', val_accuracy)
    best_accuracy = val_accuracy

    # Save weights to disk
    model.save_weights('../content/drive/My Drive/Colab Notebooks/Assignment-13_weights.h5')
  else:
    print('best accuracy: ', best_accuracy)

  if val_accuracy >= 0.93:
    print('Validation Accuracy 93% reached. Terminating training')
    break


epoch: 1 lr: 0.00016666666666666666 train loss: 0.779953042678833 train acc: 0.72602 val loss: 0.7701634453773498 val acc: 0.7478 time: 68.36180925369263
Best accuracy improved from  0.0  to:  0.7478



epoch: 2 lr: 0.0003333333333333333 train loss: 0.5314159671783447 train acc: 0.8145 val loss: 0.8127576275825501 val acc: 0.7444 time: 135.81914353370667
best accuracy:  0.7478



epoch: 3 lr: 0.0005 train loss: 0.5600657135009766 train acc: 0.80462 val loss: 0.6526333387851715 val acc: 0.7717 time: 202.8544135093689
Best accuracy improved from  0.7478  to:  0.7717



epoch: 4 lr: 0.0006666666666666666 train loss: 0.4581666399383545 train acc: 0.84054 val loss: 0.4103937803030014 val acc: 0.8632 time: 271.1376271247864
Best accuracy improved from  0.7717  to:  0.8632



epoch: 5 lr: 0.0008333333333333333 train loss: 0.36969300964355467 train acc: 0.87176 val loss: 0.6495091061592102 val acc: 0.7844 time: 339.22347712516785
best accuracy:  0.8632


...


epoch: 100 lr: 0.008333333333333333 train loss: 0.020430292186979206 train acc: 0.99306 val loss: 0.5973513813018799 val acc: 0.9028 time: 6741.886249303818
best accuracy:  0.9299



epoch: 101 lr: 0.008291666666666668 train loss: 0.026630837621986866 train acc: 0.99072 val loss: 0.37783182344436644 val acc: 0.9277 time: 6808.462944984436
best accuracy:  0.9299



epoch: 102 lr: 0.00825 train loss: 0.010968570615574718 train acc: 0.99622 val loss: 0.39509891586303714 val acc: 0.9268 time: 6874.984485626221
best accuracy:  0.9299



epoch: 103 lr: 0.008208333333333333 train loss: 0.02504728530958295 train acc: 0.9916 val loss: 0.40589174342155454 val acc: 0.9189 time: 6942.009469747543
best accuracy:  0.9299



epoch: 104 lr: 0.008166666666666668 train loss: 0.016855953997150064 train acc: 0.99436 val loss: 0.36207289400100706 val acc: 0.9301 time: 7008.922130823135
Best accuracy improved from  0.9299  to:  0.9301
Validation Accuracy 93% reached. Terminating training
